In [ ]:
%pip install olefile


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import olefile

path = "../data/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp"
f = olefile.OleFileIO(path)

In [19]:
encoded_txt = f.openstream("PrvText").read()
text1 = encoded_txt.decode("utf-16",errors="ignore")
print(text1)



<><>
<2024년 ｢벤처확인종합관리시스템 기능 고도화｣ 용역사업 (복수의결권주식, 스톡옵션, 성과조건부주식) - 제안요청서>
<><>

2024. 03.


<><<><><><><목  차><><><><>><>
<><>
< 1. 추진개요  3      2. 추진방안  5      3. 추진내용  9      4. 제안요청내용  24      5. 입찰관련사항  78      6. 제안서작성요령  82   7. 별지서식 및 붙임  94  >
<Ⅰ. 추진개요>

<1><><추진배경 및 방향>

□ 「벤처기업육성에 관한 특별조치법」(이하 ‘벤처기업법’) 복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관, 신규 구축업무를 本 과업에서 추진

  ⚬  (복수의결권주식*) 벤처기업법 제16조의11에 따라 발행된 복수의결권주식 보고 업무처리 시스템 구축

      * 복수의결권이란? 모든 주주는 1주당 하나의 의결권을 갖는 주주평등원칙(상법)과 별도로 비상장 벤처기업 창업자에게만 1주당 최대 10배의 의결권 행사를 부여하는 제도 

  ⚬  (스톡옵션*) 벤처기업법 제16조의3에 따라 부여된 벤처기업 스톡옵션 부여, 취소·철회 신고 및 업무시스템 구축

     * 스톡옵션이란? 비상장 벤처기업 임·직원 및 기업 성장에 기여한 자에게 미리 정한 가격(행사가격)으로 신주를 인수하거나 자기의 주식을 매수할 수 있는 권리 혹은 주식의 시가와 행사가격의 차액을 청구할 수 있는 권리를 부여하는 제도

     -  중소벤처24 內 스톡옵션 신청 기능 이관을 추진하며, 과거 데이터 정제 및 적재 등을 통한 데이터 이관 추진

  ⚬  (성과조건부주식*) 벤처기업법 제16조의19에 따라 성과조건부주식교부계약의 신고 및 업무처리 시스템 구축

     * 성과조건부주식이란? 비상장 벤처기업 임ㆍ직원 및 특정인에게 자기주식 부여를 약속하되, 일정 재직기간 및 조건을 충족해야 주식이 부여 대상자에게 귀속되도록 제한을 둔 


In [45]:
import olefile
import zlib
import struct
import binascii
import re

class HWPExtractor(object):
    FILE_HEADER_SECTION = "FileHeader"
    HWP_SUMMARY_SECTION = "\x05HwpSummaryInformation"
    SECTION_NAME_LENGTH = len("Section")
    BODYTEXT_SECTION = "BodyText"
    HWP_TEXT_TAGS = [67]

    def __init__(self, filename):
        self._ole = self.load(filename)
        self._dirs = self._ole.listdir()

        self._valid = self.is_valid(self._dirs)
        if (self._valid == False):
            raise Exception("Not Valid HwpFile")
        
        self._compressed = self.is_compressed(self._ole)
        self.text = self._get_text()

    def load(self, filename):
        return olefile.OleFileIO(filename)

    def is_valid(self, dirs):
        if [self.FILE_HEADER_SECTION] not in dirs:
            return False

        return [self.HWP_SUMMARY_SECTION] in dirs

    def is_compressed(self, ole):
        header = self._ole.openstream("FileHeader")
        header_data = header.read()
        return (header_data[36] & 1) == 1

    def get_body_sections(self, dirs):
        m = []
        for d in dirs:
            if d[0] == self.BODYTEXT_SECTION:
                m.append(int(d[1][self.SECTION_NAME_LENGTH:]))

        return ["BodyText/Section"+str(x) for x in sorted(m)]

    def get_text(self):
        return self.text

    def _get_text(self):
        sections = self.get_body_sections(self._dirs)
        text = ""
        for section in sections:
            text += self.get_text_from_section(section)
            text += "\n"

        self.text = text
        return self.text

    def get_text_from_section(self, section):
        bodytext = self._ole.openstream(section)
        data = bodytext.read()

        unpacked_data = zlib.decompress(data, -15) if self.is_compressed else data
        size = len(unpacked_data)

        i = 0

        text = ""
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            level = (header >> 10) & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in self.HWP_TEXT_TAGS:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                text += rec_data.decode('utf-16')
                text += "\n"

            i += 4 + rec_len

        return text
        

def get_text(filename):
    hwp = HWPExtractor(filename) 
    text = hwp.get_text()

    def clean_text(raw_text):
        # 1. 줄바꿈 정리 (\x0b 같은 제어 문자 제거)
        text = raw_text.replace('\x0b', '\n')
        
        # 2. 깨진 문자 제거 (예: ȃ, 瑢, 砼, 牠, 沈 등)
        text = re.sub(r'[^\uAC00-\uD7A3\u3131-\u318E\u1100-\u11FF\u0000-\u007F\s.,;:()\[\]\-~"\'!?0-9A-Za-z\n]', '', text)

        # 3. 제어 문자 제거 (ASCII 0~31 + 127)
        text = re.sub(r"[\x00-\x1F\x7F]", "", text)

        # 4. 여러 개의 공백/줄바꿈을 하나로 정리
        text = re.sub(r'\n{2,}', '\n\n', text)  # 줄바꿈 2번 이상 → 2번
        text = re.sub(r'[ \t]+', ' ', text)     # 탭, 다중 공백 → 공백 1개

        # 5. 앞뒤 공백 제거
        text = text.strip()
        
        return text
    
    text = clean_text(text)


    return text

def get_chunked_text(text, chunk_size=1000, chunk_overlap=100):
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,       # 각 청크 최대 길이
        chunk_overlap=chunk_overlap,    # 청크 간 겹치는 길이
        separators=["\n\n", "\n", ".", " ", ""]  # 우선순위에 따른 구분자
    )

    # 3. 청크 생성
    chunks = text_splitter.split_text(text)
    return chunks

In [51]:
text = get_text("../data/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp")
chunks = get_chunked_text(text, chunk_size=1000, chunk_overlap=100)
print(f"전체 텍스트")
print(text)
print(" ")
print(" ")
print(f"청크 개수: {len(chunks)}")
print(" ")
print(" ")
for i in range(10):
    print(f"{i+1} 번 청크:\n{chunks[i]}")
    print("-" * 50)

전체 텍스트
2024년 벤처확인종합관리시스템 기능 고도화 용역사업(복수의결권주식, 스톡옵션, 성과조건부주식) -제안요청서2024. 03.목 차 1. 추진개요 3 2. 추진방안 5 3. 추진내용 9 4. 제안요청내용 24 5. 입찰관련사항 78 6. 제안서작성요령 82 7. 별지서식 및 붙임 94. 추진개요1추진배경 및 방향 벤처기업육성에 관한 특별조치법(이하 벤처기업법) 복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관, 신규 구축업무를 과업에서 추진 (복수의결권주식*) 벤처기업법 제16조의11에 따라 발행된 복수의결권주식 보고 업무처리 시스템 구축 * 복수의결권이란? 모든 주주는 1주당 하나의 의결권을 갖는 주주평등원칙(상법)과 별도로 비상장 벤처기업 창업자에게만 1주당 최대 10배의 의결권 행사를 부여하는 제도 (스톡옵션*) 벤처기업법 제16조의3에 따라 부여된 벤처기업 스톡옵션 부여, 취소철회 신고 및 업무시스템 구축 * 스톡옵션이란? 비상장 벤처기업 임직원 및 기업 성장에 기여한 자에게 미리 정한 가격(행사가격)으로 신주를 인수하거나 자기의 주식을 매수할 수 있는 권리 혹은 주식의 시가와 행사가격의 차액을 청구할 수 있는 권리를 부여하는 제도 -중소벤처24 스톡옵션 신청 기능 이관을 추진하며, 과거 데이터 정제 및 적재 등을 통한 데이터 이관 추진 (성과조건부주식*) 벤처기업법 제16조의19에 따라 성과조건부주식교부계약의 신고 및 업무처리 시스템 구축 * 성과조건부주식이란? 비상장 벤처기업 임ㆍ직원 및 특정인에게 자기주식 부여를 약속하되, 일정 재직기간 및 조건을 충족해야 주식이 부여 대상자에게 귀속되도록 제한을 둔 주식 복수의결권주식 발행 보고 및 스톡옵션 신고기능은 기존 시스템 내 벤처확인 이력정보와 신고자(벤처기업)의 정보와 연동2사업개요 (사 업 명) 2024년 벤처확인종합관리시스템 기능 고도화 용역사업 -복수의결권주식, 스톡옵션, 성과조건부주식교부 기능 구축 및 고도화 (사업기간) 계약일로부터 1

In [21]:
%pip install -qU langchain-text-splitters langchain-community langgraph langchain-huggingface faiss-cpu bitsandbytes accelerate pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
